In [1]:
import cptac
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statistics

In [2]:
def get_gene_list(tissue):
    #Returns a list of the genes that are in both proteomics and transcriptomics
    gene_list = []
    prot = tissue.get_proteomics()
    if isinstance(prot.columns, pd.MultiIndex):
        prot = prot.columns.get_level_values(0)
    trans = tissue.get_transcriptomics()
    if isinstance(trans.columns, pd.MultiIndex):
        trans = trans.columns.get_level_values(0)
    for i in prot:
        if i in trans and i not in gene_list:
            gene_list.append(i)
    return gene_list

In [18]:
def correlation_list(tissue, gene_list, valid_sample_number = 30, tissue_type = "both"):
    #Returns a list of lists containing the name of the gene and the correlation
    corr_list = []
    pval_list = []
    prot = tissue.get_proteomics(tissue_type)
    if isinstance(prot.columns, pd.MultiIndex):
        prot = cptac.utils.reduce_multiindex(prot, levels_to_drop = "Database_ID")
    
    trans = tissue.get_transcriptomics(tissue_type)
    if isinstance(trans.columns, pd.MultiIndex):
        trans = cptac.utils.reduce_multiindex(trans, levels_to_drop = "Database_ID")
        
    prot_index_values = list(prot.index.values)
    trans_index_values = list(trans.index.values)
    

    for found_index in prot_index_values:
        if found_index not in trans_index_values:
            prot = prot.drop(index=found_index)
    for found_index in trans_index_values:
        if found_index not in prot_index_values:
            trans = trans.drop(index=found_index)
        
    for gene in gene_list:
        prot_measurements = prot[gene]
        prot_count = len(prot_measurements.dropna())
        
        trans_measurements = trans[gene]
        trans_count = len(trans_measurements.dropna())
        
        if prot_count < valid_sample_number or trans_count < valid_sample_number:
            continue
            
        nan_indices = set()
        prot_indices = list(prot_measurements.index.values)
        trans_indices = list(trans_measurements.index.values)
        
        #Here we are only currently taking the first column of multi_indices
        if isinstance(prot_measurements, pd.core.frame.DataFrame): 
            prot_measurements = prot_measurements.iloc[:,0]
        if isinstance(trans_measurements, pd.core.frame.DataFrame):
            trans_measurements = trans_measurements.iloc[:,0]
        for i in range(len(prot_measurements)):
            if math.isnan(prot_measurements[i]):
                nan_indices.add(prot_indices[i])
                
        for i in range(len(trans_measurements)):
            if math.isnan(trans_measurements[i]):
                nan_indices.add(trans_indices[i])
        
        nan_indices = list(nan_indices)
        prot_measurements = prot_measurements.drop(nan_indices)
        trans_measurements = trans_measurements.drop(nan_indices)
        
        correlation,pval = scipy.stats.pearsonr(prot_measurements, trans_measurements)
        if math.isnan(correlation):
            continue
        corr_list.append([gene,correlation])
        pval_list.append([gene, pval])
    return corr_list, pval_list

In [4]:
def ret_list(li):
    #Returns a list of correlations from all genes
    ret_li = []
    for i in li:
        ret_li.append(i[1])   
    return ret_li

In [5]:
# cptac.download("brca")
# cptac.download("ccrcc")
# cptac.download("colon")
# cptac.download("endometrial")
# cptac.download("gbm")
# cptac.download("luad")
# cptac.download("ovarian")
# cptac.download("hnscc")
# cptac.download("lscc")
brca = cptac.Brca()
ccrcc = cptac.Ccrcc()
colon = cptac.Colon()
en = cptac.Endometrial()
gbm = cptac.Gbm()
luad = cptac.Luad()
ovarian = cptac.Ovarian()
hnscc  = cptac.Hnscc()
lscc = cptac.Lscc()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 14)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 15)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 17)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 18)


In [43]:
brca_gene_list = get_gene_list(brca)
ccrcc_gene_list = get_gene_list(ccrcc)
colon_gene_list = get_gene_list(colon)
gbm_gene_list = get_gene_list(gbm)
luad_gene_list = get_gene_list(luad)
ovarian_gene_list = get_gene_list(ovarian)
en_gene_list = get_gene_list(en)
hnscc_gene_list = get_gene_list(hnscc)
lscc_gene_list = get_gene_list(lscc)

In [44]:
brca_corr, brca_pval = correlation_list(brca, brca_gene_list, tissue_type = "tumor")
ccrcc_corr, ccrcc_pval = correlation_list(ccrcc, ccrcc_gene_list, tissue_type = "tumor")
colon_corr, colon_pval = correlation_list(colon, colon_gene_list, tissue_type = "tumor")
en_corr, en_pval = correlation_list(en, en_gene_list, tissue_type = "tumor")
gbm_corr, gbm_pval = correlation_list(gbm, gbm_gene_list, tissue_type = "tumor")
luad_corr, luad_pval = correlation_list(luad,luad_gene_list, tissue_type = "tumor")
ovarian_corr, ovarian_pval = correlation_list(ovarian, ovarian_gene_list, tissue_type = "tumor")
hnscc_corr, hnscc_pval = correlation_list(hnscc, hnscc_gene_list, tissue_type = "tumor")
lscc_corr, lscc_pval = correlation_list(lscc, lscc_gene_list, tissue_type = "tumor")

cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)
/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
cptac warning: Due to dropping the specified levels, dataframe now has 2495 duplicated column headers. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 11)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/s

In [76]:
genes = brca_gene_list + ccrcc_gene_list + colon_gene_list + en_gene_list + gbm_gene_list+ luad_gene_list + ovarian_gene_list + hnscc_gene_list + lscc_gene_list
common_genes = []
for i in genes:
    if i not in common_genes:
        common_genes.append(i)
common_genes = sorted(common_genes)

In [77]:
# for item in brca_corr:
#     if item[0] == "RGMA":
#         print("Here:")
#         print(item[1])

In [78]:
# for item in brca_corr:
#     print(item)

In [79]:
def fill_genes_nan(all_genes, tissue_corr):
    tissue_genes = []
    for i, j  in tissue_corr:
        tissue_genes.append(i)
        
    for i in all_genes:
        if i not in tissue_genes:
            tissue_corr.append([i, np.nan])
    end_list = sorted(tissue_corr)
           
    return end_list

In [80]:
# brca_list_1 = fill_genes_nan(common_genes, brca_corr)
# ccrcc_list_1 = fill_genes_nan(common_genes, ccrcc_corr)

In [81]:
# print(brca_list_1[13478])
# print(ccrcc_list_1[13478])
# print(brca_list_1[10379])
# print(ccrcc_list_1[10379])

In [82]:
# for i in range(len(brca_list_1)):
#     if brca_list_1[i][0] == "RGMA":
#         print("i = ",i)

In [83]:
# # print(brca_list_corr)
# # print(len(lscc_list_corr))
# # print(len(common_genes))
# common_genes_2 = sorted(common_genes)
# print(common_genes[13478])
# print(common_genes_2[13478])

In [84]:
brca_list_corr = ret_list(fill_genes_nan(common_genes, brca_corr))
ccrcc_list_corr = ret_list(fill_genes_nan(common_genes, ccrcc_corr))
colon_list_corr = ret_list(fill_genes_nan(common_genes, colon_corr))
en_list_corr = ret_list(fill_genes_nan(common_genes, en_corr))
gbm_list_corr = ret_list(fill_genes_nan(common_genes, gbm_corr))
luad_list_corr = ret_list(fill_genes_nan(common_genes, luad_corr))
ovarian_list_corr = ret_list(fill_genes_nan(common_genes, ovarian_corr))
hnscc_list_corr = ret_list(fill_genes_nan(common_genes, hnscc_corr))
lscc_list_corr = ret_list(fill_genes_nan(common_genes, lscc_corr))

mean_list = []
std_list = []
sigma_list = []
range_list = []
def minmax(val_list):
    if len(val_list) == 0:
        return np.nan, np.nan
    else:
        min_val = min(val_list)
        max_val = max(val_list)

    return (min_val, max_val)
# curr_gene = False
print(len(common_genes))
for i in range(len(common_genes)):
#     if common_genes[i] == "RGMA":
#         curr_gene = True
#         print("Found RGMA")
#         print(brca_list_corr[i])
#         print("i = ",i)
    row = list(filter(lambda x:not math.isnan(x),[brca_list_corr[i], ccrcc_list_corr[i], colon_list_corr[i], 
             en_list_corr[i],gbm_list_corr[i],luad_list_corr[i],ovarian_list_corr[i],
             hnscc_list_corr[i],lscc_list_corr[i]]))#Creates a list without Nan on each row

    counter = len(row)
    if counter == 0:
        counter = 1
        
    ###Mean calculation
    mean = sum(row)/counter
    mean_list.append(mean)
    
    ###Standard Deviation calculation
    std = 0
    if len(row) < 2:
        std = np.nan    
    else:
        std = statistics.stdev(row)
    std_list.append(std)
    
    ### 3 sigma calculation
    neg_sigma = mean - 2.5 * std
    pos_sigma = mean + 2.5 * std
    
    for i in row:
        sigma_bool = False
        if i > pos_sigma or i < neg_sigma:
            sigma_bool = True
            break
    sigma_list.append(sigma_bool)
    
    ###Calculating range
    min_value, max_value = minmax(row)
    range_list.append(max_value - min_value)
    
    
correlation_columns = {'BRCA': (brca_list_corr),'CCRCC': (ccrcc_list_corr), 
       'Colon': (colon_list_corr), 'Endometrial': (en_list_corr),
       'GBM': (gbm_list_corr), 'LUAD': (luad_list_corr), 'Ovarian': (ovarian_list_corr),
        'HNSCC' : (hnscc_list_corr), 'LSCC' : (lscc_list_corr), 'Mean' : mean_list, 'Stnd Deviation' :std_list, '+/- 3 sigma' : sigma_list, 'Range' : range_list }

correlation_df = pd.DataFrame(correlation_columns , columns = ['BRCA', 'CCRCC', 'Colon', 'Endometrial', 'GBM', 'LUAD', 'Ovarian', 'HNSCC', 'LSCC', 'Mean', 'Stnd Deviation', '+/- 3 sigma', 'Range'], index=common_genes)
correlation_df

14636


,BRCA,CCRCC,Colon,Endometrial,GBM,LUAD,Ovarian,HNSCC,LSCC,Mean,Stnd Deviation,+/- 3 sigma,Range
A1BG,0.131286,-0.059781,NaN,0.386703,-0.200579,-0.067920,NaN,0.112133,0.165873,0.066816,0.193232,False,0.587282
A1CF,NaN,0.892757,0.698025,NaN,NaN,NaN,NaN,NaN,NaN,0.795391,0.137696,False,0.194732
A2M,0.422497,0.010289,-0.122255,0.024022,0.312922,0.381025,0.140341,0.201083,0.355647,0.191730,0.191378,False,0.544752
A2ML1,0.713289,NaN,NaN,0.835080,NaN,NaN,0.531368,0.862612,0.817462,0.751962,0.135646,False,0.331245
A4GALT,NaN,NaN,NaN,0.486499,NaN,NaN,NaN,NaN,0.286010,0.386255,0.141767,False,0.200489
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.200364,0.124010,NaN,0.146719,0.185779,0.212842,NaN,0.488809,0.583187,0.277387,0.181339,False,0.459178
ZYG11B,-0.029503,0.241315,NaN,0.231203,0.458107,0.556451,0.371712,0.324304,0.387166,0.317594,0.176623,False,0.585954
ZYX,0.546670,0.217698,0.144728,0.532447,0.628549,0.414920,0.300631,0.544006,0.522374,0.428002,0.168982,False,0.483821
ZZEF1,0.280712,0.197289,0.087799,0.299816,0.424073,0.679603,0.218811,0.555744,0.630379,0.374914,0.207991,False,0.591803


In [85]:
correlation_df.loc[correlation_df['+/- 3 sigma'] == True]

,BRCA,CCRCC,Colon,Endometrial,GBM,LUAD,Ovarian,HNSCC,LSCC,Mean,Stnd Deviation,+/- 3 sigma,Range
CFL2,0.698358,0.739900,0.644461,0.751337,0.623642,0.674934,-0.012246,0.816952,0.577462,0.612755,0.245270,True,0.829198
CMPK2,0.804784,0.719638,0.405478,0.813236,0.832938,0.802153,0.832165,0.902028,0.857366,0.774421,0.146731,True,0.496550
CRISPLD2,0.675279,0.557736,0.295992,0.639656,0.605271,0.636899,0.668486,0.621525,0.662528,0.595930,0.118190,True,0.379287
ECT2,0.587818,-0.139274,0.612958,0.856567,0.699655,0.827534,0.710159,0.798158,0.879742,0.648146,0.312883,True,1.019016
EIF4ENIF1,0.627583,0.609696,-0.120662,0.656449,0.712156,0.635596,0.591006,0.720241,0.651353,0.564824,0.260596,True,0.840903
EIF5B,0.579179,0.493952,0.536926,0.492747,0.149446,0.603375,0.555367,0.498909,0.602994,0.501433,0.139173,True,0.453929
EPB41L2,0.698096,-0.010817,0.697713,0.907437,0.698162,0.768173,0.703405,0.815509,0.701344,0.664336,0.263358,True,0.918254
FAM107B,0.880467,0.793304,0.352687,0.738770,0.724647,0.862736,0.780461,0.811781,0.864860,0.756635,0.161158,True,0.527780
FAS,0.738086,0.774621,0.733408,0.731046,0.688361,0.765613,0.263238,0.865481,0.784079,0.704882,0.172677,True,0.602243
FUCA1,0.696446,0.681138,0.382992,0.622044,0.646362,0.707473,0.717193,0.729365,0.704841,0.654206,0.107343,True,0.346373


In [25]:
brca_list_pval = (ret_list(fill_genes_nan(common_genes, brca_pval)))
ccrcc_list_pval = (ret_list(fill_genes_nan(common_genes, ccrcc_pval)))
colon_list_pval = (ret_list(fill_genes_nan(common_genes, colon_pval)))
en_list_pval = (ret_list(fill_genes_nan(common_genes, en_pval)))
gbm_list_pval = (ret_list(fill_genes_nan(common_genes, gbm_pval)))
luad_list_pval = (ret_list(fill_genes_nan(common_genes, luad_pval)))
ovarian_list_pval = (ret_list(fill_genes_nan(common_genes, ovarian_pval)))
hnscc_list_pval = (ret_list(fill_genes_nan(common_genes, hnscc_pval)))
lscc_list_pval = (ret_list(fill_genes_nan(common_genes, lscc_pval)))


pval_columns = {'BRCA': (brca_list_pval),
        'CCRCC': (ccrcc_list_pval), 'Colon': (colon_list_pval), 'Endometrial': (en_list_pval),
       'GBM': (gbm_list_pval), 'LUAD': (luad_list_pval), 'Ovarian': (ovarian_list_pval), 'HNSCC' : (hnscc_list_pval), 'LSCC' : (lscc_list_pval)}

pval_df= pd.DataFrame(pval_columns , columns = ['BRCA', 'CCRCC', 'Colon', 'Endometrial', 'GBM', 'LUAD', 'Ovarian', 'HNSCC', 'LSCC'], index=common_genes)
pval_df

,BRCA,CCRCC,Colon,Endometrial,GBM,LUAD,Ovarian,HNSCC,LSCC
A1BG,1.657241e-01,5.350095e-01,NaN,1.084161e-04,4.651711e-02,4.807936e-01,NaN,2.456948e-01,9.082049e-02
A2M,NaN,3.432584e-39,5.325574e-15,NaN,NaN,NaN,NaN,NaN,NaN
A2ML1,1.249853e-06,9.150454e-01,2.353794e-01,8.172572e-01,1.612770e-03,4.015329e-05,0.208548,3.602656e-02,1.703835e-04
AAAS,1.373971e-17,NaN,NaN,9.999470e-23,NaN,NaN,0.000013,1.908976e-33,6.589307e-27
AACS,NaN,NaN,NaN,2.594752e-05,NaN,NaN,NaN,NaN,1.805955e-02
...,...,...,...,...,...,...,...,...,...
ZNF782,4.909360e-02,2.958874e-01,NaN,1.559574e-01,7.958310e-02,5.049816e-02,NaN,3.776369e-07,3.654355e-10
ZNF85,7.470119e-01,1.109573e-02,NaN,2.417794e-02,1.861457e-06,2.757689e-10,0.000585,5.811263e-04,3.793685e-05
ZNF880,7.319264e-11,2.569032e-02,1.594617e-01,2.798595e-08,3.274769e-12,6.578244e-06,0.006064,9.742303e-10,7.881491e-09
ZNF91,1.736969e-03,3.883803e-02,3.949742e-01,3.159430e-03,1.218953e-05,3.279695e-16,0.048267,3.547520e-10,3.444445e-13


In [28]:
en.get_transcriptomics('DOK2')

Name,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,4.02,2.16,3.27,13.39,5.88,6.79,1.55,0.97,10.34,1.96,...,11.06,10.73,8.40,9.78,10.88,5.93,11.52,10.23,11.50,11.47
C3L-00008,4.81,2.21,4.86,13.24,5.93,6.33,0.93,0.00,10.83,0.00,...,10.87,11.43,8.39,9.14,10.38,7.25,11.64,10.64,11.26,11.57
C3L-00032,6.24,6.43,3.68,14.32,6.53,9.42,2.79,0.00,10.98,2.13,...,10.06,10.13,8.35,9.27,10.46,6.85,11.60,10.21,11.51,11.09
C3L-00090,5.31,4.87,5.59,13.77,6.35,4.22,2.97,0.00,8.68,1.98,...,10.29,10.41,9.10,9.59,10.15,7.89,11.90,10.21,11.34,11.51
C3L-00098,9.84,8.83,7.00,13.12,6.49,6.83,1.80,0.00,11.42,3.28,...,10.36,11.24,8.60,9.44,11.80,9.32,11.97,9.77,11.37,12.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-00333.N,7.05,7.26,3.71,15.84,8.22,5.70,3.57,0.00,9.03,2.21,...,8.75,6.07,10.02,10.63,11.43,7.72,12.62,11.85,12.10,11.62
C3N-00383.N,6.58,6.92,2.29,16.02,8.88,5.86,4.44,1.84,9.47,3.66,...,8.63,7.20,9.73,10.14,11.36,7.15,12.60,11.63,12.02,11.96
C3N-00858.N,7.36,7.55,2.99,15.17,8.68,5.64,3.65,2.16,8.59,1.73,...,8.69,5.99,9.89,10.05,11.29,6.96,12.72,11.02,12.03,11.68


In [30]:
brca_prot = brca.get_proteomics()
ccrcc_prot = ccrcc.get_proteomics()
colon_prot = colon.get_proteomics()

In [31]:
brca_prot['AMD1']

KeyError: 'AMD1'

In [38]:
# brca_prot
new = cptac.utils.reduce_multiindex(brca_prot, levels_to_drop = "Database_ID")
# cptac.utils.reduce_multiindex(trans, levels_to_drop = "Database_ID")
for col in new.columns:
    print(col)

cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


A1BG
A2M
A2ML1
AAAS
AACS
AADAT
AAED1
AAGAB
AAK1
AAMDC
AAMP
AAR2
AARS
AARS2
AARSD1
AASDHPPT
AASS
AATF
ABAT
ABCA1
ABCA12
ABCA13
ABCA2
ABCA3
ABCA6
ABCA8
ABCB1
ABCB10
ABCB5
ABCB6
ABCB7
ABCB7
ABCB8
ABCC1
ABCC10
ABCC11
ABCC2
ABCC3
ABCC4
ABCC5
ABCD1
ABCD3
ABCE1
ABCF1
ABCF2
ABCF3
ABCG1
ABHD10
ABHD11
ABHD12
ABHD13
ABHD14B
ABHD16A
ABHD17A
ABHD17B
ABHD17C
ABHD3
ABHD4
ABHD5
ABHD6
ABI1
ABI2
ABI3
ABI3BP
ABL1
ABLIM1
ABLIM2
ABLIM3
ABLIM3
ABR
ABRACL
ABT1
ABTB1
ABTB2
ACAA1
ACAA2
ACACA
ACACB
ACAD10
ACAD11
ACAD8
ACAD9
ACADL
ACADM
ACADS
ACADSB
ACADVL
ACAN
ACAP1
ACAP2
ACAP3
ACAT1
ACAT2
ACBD3
ACBD5
ACBD6
ACBD7
ACCS
ACD
ACE
ACE2
ACIN1
ACIN1
ACLY
ACLY
ACO1
ACO2
ACOT1
ACOT11
ACOT13
ACOT2
ACOT4
ACOT7
ACOT8
ACOT9
ACOX1
ACOX1
ACOX3
ACP1
ACP1
ACP2
ACP2
ACP5
ACP6
ACPP
ACSF2
ACSF3
ACSL1
ACSL3
ACSL4
ACSL5
ACSM1
ACSM3
ACSS1
ACSS2
ACSS3
ACTA1
ACTA2
ACTB
ACTBL2
ACTG1
ACTG2
ACTL6A
ACTL8
ACTN1
ACTN1
ACTN2
ACTN3
ACTN4
ACTN4
ACTR1A
ACTR1B
ACTR2
ACTR3
ACTR3B
ACTR3C
ACTR5
ACTR6
ACTR8
ACVR1
ACY1
ACYP1
ACYP2
ADA
ADAL
ADAM10
ADAM

COL17A1
COL18A1
COL1A1
COL1A2
COL21A1
COL22A1
COL24A1
COL27A1
COL28A1
COL2A1
COL3A1
COL4A1
COL4A2
COL4A3
COL4A3BP
COL4A4
COL4A5
COL4A6
COL5A1
COL5A2
COL5A3
COL6A1
COL6A2
COL6A2
COL6A3
COL6A3
COL6A5
COL6A6
COL7A1
COL8A1
COL8A2
COL9A1
COL9A2
COL9A3
COLEC10
COLEC12
COLGALT1
COLGALT2
COMMD1
COMMD10
COMMD2
COMMD3
COMMD3-BMI1
COMMD4
COMMD5
COMMD6
COMMD7
COMMD8
COMMD9
COMP
COMT
COMTD1
COPA
COPB1
COPB2
COPE
COPG1
COPG2
COPRS
COPS2
COPS2
COPS4
COPS5
COPS6
COPS7A
COPS7B
COPS8
COPS9
COPZ1
COPZ2
COQ10B
COQ3
COQ5
COQ6
COQ7
COQ8A
COQ8B
COQ9
CORO1A
CORO1B
CORO1C
CORO2A
CORO2B
CORO6
CORO7
CORO7-PAM16
COTL1
COX11
COX15
COX17
COX19
COX20
COX4I1
COX5A
COX5B
COX6A1
COX6B1
COX6C
COX7A1
COX7A2
COX7A2L
COX7B
COX7C
COX8A
CP
CPA3
CPA4
CPB1
CPB2
CPD
CPE
CPEB2
CPEB4
CPLX2
CPM
CPN1
CPN2
CPNE1
CPNE2
CPNE3
CPNE8
CPOX
CPPED1
CPQ
CPS1
CPSF1
CPSF3
CPSF4
CPSF6
CPSF7
CPT1A
CPT2
CPTP
CPVL
CPXM1
CPXM2
CPZ
CRABP1
CRABP2
CRACR2A
CRACR2B
CRADD
CRAMP1
CRAT
CRBN
CRCP
CREB1
CREB3L4
CREB5
CREBBP
CREG1
CRELD1
CRELD2
CRIM1
CRIP1
C

GBF1
GBP1
GBP2
GBP3
GBP4
GBP5
GBP7
GC
GCA
GCAT
GCC1
GCC2
GCDH
GCFC2
GCH1
GCHFR
GCLC
GCLM
GCN1
GCOM1
GCSH
GDAP1
GDAP2
GDE1
GDF15
GDI1
GDI2
GDPD1
GDPD3
GDPGP1
GEM
GEMIN2
GEMIN4
GEMIN5
GEMIN6
GEMIN7
GEMIN8
GEN1
GET4
GFAP
GFER
GFM1
GFM2
GFOD1
GFOD2
GFPT1
GFPT2
GFRA1
GFRA1
GGA1
GGA2
GGA3
GGACT
GGCT
GGCX
GGH
GGNBP2
GGPS1
GGT1
GGT5
GGT7
GHDC
GHITM
GID4
GID8
GIGYF1
GIGYF2
GIMAP1
GIMAP1-GIMAP5
GIMAP2
GIMAP4
GIMAP6
GIMAP7
GIMAP8
GINS1
GINS2
GINS3
GINS4
GIPC1
GIPC2
GIPC3
GIT1
GIT1
GIT2
GIT2
GJA1
GK
GK5
GKAP1
GLA
GLB1
GLB1L
GLCCI1
GLCE
GLDC
GLE1
GLG1
GLI3
GLIPR1
GLIPR2
GLMN
GLMP
GLO1
GLOD4
GLRX
GLRX2
GLRX3
GLRX5
GLS
GLS
GLT8D1
GLT8D2
GLTP
GLTSCR1
GLTSCR1L
GLTSCR2
GLUD1
GLUD2
GLUL
GLYAT
GLYATL1
GLYATL2
GLYR1
GM2A
GMDS
GMEB1
GMEB1
GMEB2
GMFB
GMFG
GMIP
GMNN
GMPPA
GMPPB
GMPR
GMPR2
GMPR2
GMPS
GNA11
GNA12
GNA13
GNA15
GNAI1
GNAI2
GNAI3
GNAO1
GNAQ
GNAS
GNAS
GNAT3
GNAZ
GNB1
GNB1L
GNB2
GNB4
GNE
GNG10
GNG11
GNG12
GNG2
GNG4
GNG5
GNG7
GNL1
GNL2
GNL3
GNL3L
GNMT
GNPAT
GNPDA1
GNPDA2
GNPNAT1
GNPTAB
GNPTG
GNS
GOLGA

MT1G
MT1X
MT2A
MTA1
MTA2
MTA3
MTAP
MTCH1
MTCH2
MTCH2
MTCL1
MTDH
MTERF1
MTERF3
MTERF4
MTF1
MTF2
MTFMT
MTFR1
MTFR1L
MTG1
MTG2
MTHFD1
MTHFD2
MTHFD2L
MTHFR
MTHFS
MTHFSD
MTHFSD
MTIF2
MTIF3
MTM1
MTMR1
MTMR10
MTMR11
MTMR12
MTMR14
MTMR2
MTMR2
MTMR3
MTMR4
MTMR6
MTMR7
MTMR9
MTO1
MTOR
MTPAP
MTPN
MTR
MTRF1
MTRF1L
MTRR
MTSS1
MTSS1L
MTURN
MTUS1
MTUS1
MTX1
MTX2
MTX3
MTX3
MUC1
MUC16
MUC19
MUC2
MUC5AC
MUC5B
MUC6
MUCL1
MUL1
MUM1
MUSK
MUSTN1
MUT
MUTYH
MVB12A
MVB12B
MVD
MVK
MVP
MX1
MX2
MXI1
MXRA5
MXRA7
MXRA7
MXRA7
MXRA8
MYADM
MYBBP1A
MYBPC1
MYCBP
MYCBP2
MYD88
MYD88
MYDGF
MYEF2
MYEF2
MYH10
MYH11
MYH11
MYH14
MYH7
MYH7B
MYH9
MYL12B
MYL4
MYL6
MYL6
MYL6B
MYL9
MYLK
MYLK
MYLK2
MYLK3
MYLK4
MYO10
MYO15A
MYO18A
MYO18B
MYO19
MYO1B
MYO1C
MYO1C
MYO1D
MYO1E
MYO1F
MYO1G
MYO3B
MYO5A
MYO5B
MYO5C
MYO6
MYO6
MYO7B
MYO9A
MYO9B
MYOC
MYOF
MYOF
MYOM2
MYOM3
MYOT
MYSM1
MYT1
MZB1
MZT1
MZT2A
MZT2B
N4BP1
N4BP2
N4BP2L1
N4BP2L2
N4BP3
N6AMT1
NAA10
NAA15
NAA16
NAA20
NAA25
NAA30
NAA35
NAA38
NAA40
NAA50
NAAA
NAALADL2
NAB1
NAB2
NABP2
NACA
N

RICTOR
RIDA
RIF1
RILP
RILPL1
RILPL2
RIN1
RIN2
RIN3
RING1
RINL
RINT1
RIOK1
RIOK2
RIOK3
RIPK1
RIPK2
RIPK3
RIT1
RLF
RLIM
RMDN1
RMDN2
RMDN3
RMI1
RMND1
RNASE1
RNASE3
RNASE4
RNASE6
RNASEH1
RNASEH2A
RNASEH2B
RNASEH2C
RNASEL
RNASET2
RND3
RNF10
RNF11
RNF113A
RNF114
RNF115
RNF121
RNF123
RNF126
RNF13
RNF130
RNF135
RNF138
RNF139
RNF14
RNF141
RNF146
RNF149
RNF166
RNF167
RNF168
RNF169
RNF17
RNF170
RNF181
RNF185
RNF2
RNF213
RNF214
RNF216
RNF219
RNF220
RNF25
RNF31
RNF34
RNF4
RNF40
RNF41
RNF5
RNF7
RNGTT
RNH1
RNLS
RNMT
RNPC3
RNPEP
RNPEPL1
ROBO1
ROBO2
ROCK1
ROCK2
ROGDI
ROR2
ROS1
RP1
RP11-274B21.1
RP2
RP9
RPA1
RPA2
RPA3
RPAIN
RPAP1
RPAP2
RPE
RPF1
RPF2
RPGR
RPGRIP1L
RPIA
RPL10
RPL10A
RPL10L
RPL13
RPL13A
RPL14
RPL15
RPL17-C18orf32
RPL18
RPL18A
RPL19
RPL21
RPL22
RPL22L1
RPL23
RPL23A
RPL24
RPL26
RPL26L1
RPL27
RPL27A
RPL28
RPL29
RPL3
RPL30
RPL31
RPL32
RPL34
RPL35
RPL35A
RPL36
RPL36A
RPL36AL
RPL37
RPL37A
RPL38
RPL39
RPL4
RPL5
RPL6
RPL7
RPL7A
RPL7L1
RPL8
RPL9
RPLP1
RPN1
RPN2
RPP14
RPP14_ORF1
RPP21
RPP25
RPP25L
R

UBA52
UBA6
UBA7
UBAC1
UBAC2
UBALD1
UBAP1
UBAP2
UBAP2L
UBAP2L
UBASH3B
UBE2A
UBE2B
UBE2C
UBE2D3
UBE2E1
UBE2E2
UBE2E3
UBE2F
UBE2G1
UBE2G2
UBE2H
UBE2I
UBE2J1
UBE2J2
UBE2K
UBE2K
UBE2L3
UBE2L6
UBE2M
UBE2N
UBE2O
UBE2Q1
UBE2Q2
UBE2R2
UBE2S
UBE2T
UBE2V1
UBE2V2
UBE2W
UBE2Z
UBE3A
UBE3B
UBE3C
UBE4A
UBE4B
UBFD1
UBIAD1
UBL4A
UBL5
UBL7
UBLCP1
UBN1
UBN2
UBOX5
UBP1
UBQLN1
UBQLN1
UBQLN2
UBQLN4
UBR1
UBR2
UBR3
UBR4
UBR5
UBR7
UBTD2
UBTF
UBTF
UBXN1
UBXN10
UBXN2A
UBXN2B
UBXN4
UBXN6
UBXN7
UCHL1
UCHL3
UCHL5
UCHL5
UCK1
UCK2
UCKL1
UEVLD
UFC1
UFD1L
UFL1
UFM1
UFSP1
UFSP2
UGCG
UGDH
UGGT1
UGGT2
UGP2
UGP2
UGT8
UHRF1
UHRF1BP1
UHRF1BP1L
UHRF2
UIMC1
ULK1
ULK3
ULK4
UMAD1
UMPS
UNC119
UNC119B
UNC13A
UNC13B
UNC13D
UNC45A
UNC5B
UNC93B1
UNCX
UNG
UNG
UNK
UNKL
UPF1
UPF2
UPF3A
UPF3B
UPP1
UPRT
UQCC1
UQCC2
UQCC3
UQCR10
UQCR11
UQCRB
UQCRC1
UQCRC2
UQCRFS1
UQCRH
UQCRQ
URB1
URB2
URGCP
URI1
URM1
UROD
UROS
USE1
USF1
USF2
USH2A
USMG5
USO1
USO1
USP1
USP10
USP11
USP12
USP13
USP14
USP14
USP15
USP16
USP18
USP20
USP22
USP24
USP25
USP28
USP29
